# 📦 11_load_playlist_items_snapshot_to_bigquery

### 🎯 Objetivo  

Este notebook no transforma datos.
Su única responsabilidad es: Cargar df_playlist_items_snapshot (generado en el notebook 08) hacia BigQuery como tabla histórica.

Destino:  
youtube-datasets-360.angelgarciadatablog.playlist_items_snapshot

In [1]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery


In [2]:
load_dotenv()

PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "playlist_items_snapshot"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)


Destino configurado: youtube-datasets-360.angelgarciadatablog.playlist_items_snapshot


## 🧱 Cargar snapshot desde Parquet (temporal)  

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [3]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

file_path = PROCESSED_PATH / "playlist_items_snapshot.parquet"

df_playlist_items_snapshot = pd.read_parquet(file_path)

df_playlist_items_snapshot.head()

,snapshot_date,playlist_id,video_id,position,added_at,extracted_at
0,2026-02-14,PLV4oS06_KpqbsY_I8iR4HRvb6w3vXUBIM,7bwkNrRpgw0,0,2026-01-23 06:51:06+00:00,2026-02-14 23:00:21.011925+00:00
1,2026-02-14,PLV4oS06_KpqbsY_I8iR4HRvb6w3vXUBIM,HDyKUodeuNw,1,2026-01-23 06:37:27+00:00,2026-02-14 23:00:21.011925+00:00
2,2026-02-14,PLV4oS06_KpqZGwOHo-tsdIiaZts7qaqql,Zj6uiqMvFOU,0,2026-01-17 20:02:37+00:00,2026-02-14 23:00:21.011925+00:00
3,2026-02-14,PLV4oS06_KpqZGwOHo-tsdIiaZts7qaqql,RiYjYfMTGvw,1,2026-01-11 23:05:55+00:00,2026-02-14 23:00:21.011925+00:00
4,2026-02-14,PLV4oS06_KpqZGwOHo-tsdIiaZts7qaqql,0VmI47XeOuE,2,2026-01-11 23:05:33+00:00,2026-02-14 23:00:21.011925+00:00


In [4]:
df_playlist_items_snapshot.dtypes

snapshot_date                 object
playlist_id                      str
video_id                         str
position                       int64
added_at         datetime64[us, UTC]
extracted_at     datetime64[us, UTC]
dtype: object

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 



In [5]:
from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("snapshot_date", "DATE"),
    SchemaField("playlist_id", "STRING"),
    SchemaField("video_id", "STRING"),
    SchemaField("position", "INT64"),
    SchemaField("added_at", "TIMESTAMP"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("Tabla ya existe.")
    
except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)

    table.time_partitioning = bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="snapshot_date",
    )

    client.create_table(table)
    print("Tabla creada con partición.")




Tabla creada con partición.


## 🔒 Control de idempotencia por `snapshot_date`

Antes de insertar el snapshot actual, se eliminan los registros existentes con el mismo `snapshot_date`.

Esto garantiza que el proceso sea **idempotente**:  Si el pipeline se ejecuta múltiples veces el mismo día (pruebas, re-ejecuciones o errores), el resultado final en BigQuery será siempre consistente y sin duplicados.


In [6]:
snapshot_date = df_playlist_items_snapshot["snapshot_date"].iloc[0]

delete_query = f"""
DELETE FROM `{FULL_TABLE_ID}`
WHERE snapshot_date = @snapshot_date
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter(
            "snapshot_date",
            "DATE",
            snapshot_date
        )
    ]
)

client.query(delete_query, job_config=job_config).result()

print(f"Snapshots del {snapshot_date} eliminados si existían.")

Snapshots del 2026-02-14 eliminados si existían.


## 📌 Cargar datos del parquet a big query

In [7]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE APPEND = Agrega fila nuevas
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df_playlist_items_snapshot,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Carga completada correctamente.")

/Users/angelgarciachanga/repositorios/publico/youtube-v3-data-pipeline/venv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Carga completada correctamente.
